## **0. LOAD LIBRARY**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. LOAD MODEL**

In [3]:
# --- INSTALL LIBRARIES & MOUNT DRIVE ---
# Install the core libraries and hf_transfer for fast downloads
!pip install transformers accelerate torch pillow requests hf_transfer -q

import os
from google.colab import drive
from huggingface_hub import login

# Mount Google Drive (you will be prompted to authorize)
drive.mount('/content/drive')

# Enable fast downloads and log in to Hugging Face
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'
login()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U "transformers>=4.44.0" "accelerate>=0.34.0" "safetensors>=0.4.2" "sentencepiece" "huggingface_hub>=0.23.0" -q

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, os

MODEL_ID = "OpenMeditron/Meditron3-Gemma2-2B"
DRIVE_PATH = "/content/drive/MyDrive/Meditron3-Gemma2-2B"  # change if needed

# (Colab) mount Drive if you haven’t yet
from google.colab import drive
drive.mount('/content/drive')

# Pick a safe dtype for your GPU
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16

print(f"1. Loading {MODEL_ID} ...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

print("2. Model loaded.")

# Save locally (e.g., Google Drive)
os.makedirs(DRIVE_PATH, exist_ok=True)
model.save_pretrained(DRIVE_PATH)
tokenizer.save_pretrained(DRIVE_PATH)
print(f"3. Saved to: {DRIVE_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1. Loading OpenMeditron/Meditron3-Gemma2-2B ...


`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

2. Model loaded.
3. Saved to: /content/drive/MyDrive/Meditron3-Gemma2-2B


## **2. QUICK TEST MODEL**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, os

# Path where you saved the model earlier
DRIVE_PATH = "/content/drive/MyDrive/Meditron3-Gemma2-2B"

# --- Load model & tokenizer from Drive ---
print("1. Loading model from Drive...")
tokenizer = AutoTokenizer.from_pretrained(DRIVE_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    DRIVE_PATH,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
print("2. Model loaded successfully!")

1. Loading model from Drive...


`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2. Model loaded successfully!

--- MODEL RESPONSE ---
i am a medical student asking this question for class material to answer my prof on assignment. The patient said: My head hurts a lot, I'll take paracetamol. Can I use it with Ginkgo Biloba?
Answer: No.
Explanation: Paracetamol is a safe and effective analgesic. Ginkgo Biloba is a herbal remedy that is not approved by the FDA for use in the treatment of headache. It is not known whether Ginkgo Biloba is safe or effective for this use.
Answer: No.
Explanation: Paracetamol is a safe and effective analgesic. Ginkgo Biloba is a herbal remedy that is not approved by the FDA for use in the treatment of headache. It is not known whether Ginkgo Biloba is safe or effective for this use.
Answer: No.
Explanation: Paracetamol is a safe and effective analgesic. Ginkgo Biloba is a herbal remedy that is not approved by the FDA for use in the


"i am a medical student asking this question for class material to answer my prof on assignment. The patient said: My head hurts a lot, I'll take paracetamol. Can I use it with Ginkgo Biloba?\nAnswer: No.\nExplanation: Paracetamol is a safe and effective analgesic. Ginkgo Biloba is a herbal remedy that is not approved by the FDA for use in the treatment of headache. It is not known whether Ginkgo Biloba is safe or effective for this use.\nAnswer: No.\nExplanation: Paracetamol is a safe and effective analgesic. Ginkgo Biloba is a herbal remedy that is not approved by the FDA for use in the treatment of headache. It is not known whether Ginkgo Biloba is safe or effective for this use.\nAnswer: No.\nExplanation: Paracetamol is a safe and effective analgesic. Ginkgo Biloba is a herbal remedy that is not approved by the FDA for use in the"

In [ ]:
# --- Define a helper function to ask questions ---
def ask(prompt, max_new_tokens=200, temperature=0.7, top_p=0.9): # Tăng temperature lên 0.7
    """Generate model responses for text prompts."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n--- MODEL RESPONSE ---")
    print(response)
    return response

# --- Question ---
model_answer = ask("i am a medical student asking this question for class material to answer my assignment. My head hurts a lot, I'll take paracetamol. Can I use it with Ginkgo Biloba?")
print(model_answer.split('\n')[1])


--- MODEL RESPONSE ---
i am a medical student asking this question for class material to answer my assignment. My head hurts a lot, I'll take paracetamol. Can I use it with Ginkgo Biloba?

Paracetamol is an antipyretic and analgesic drug that has been used safely in patients receiving warfarin therapy since 1984 [bib_ref] Paracetamol: safety of its routine clinical usage at low doses -a review..., Williams [/bib_Ref]. The recommended dose regimen for adults or children weighing over 60 kg are one standard-size tablet (5 g) every four hours as needed up to eight tablets per day; the equivalent dosage range is two grams divided into three equal oral dosing intervals no more than once each hour if necessary until relief from pain occurs. Children under age six years should receive half the adult daily maximum amount but not less than thirty milligrams per kilogram body weight/day unless otherwise directed by their physician due to liver function impairment.
The typical adverse effects in

## **3. LOAD BIOBERT FOR EVALUATION MEDICAL QnA QUESTIONS**

In [6]:
from sentence_transformers import SentenceTransformer
# Create the target directory in your Drive
BioBERT_BDRIVE_PATH = "/content/drive/MyDrive/BioBERT"
os.makedirs(DRIVE_PATH, exist_ok=True)

# Load BioBert model
bert_model_name = "pritamdeka/BioBert-PubMed200kRCT"
save_path = os.path.join(BioBERT_BDRIVE_PATH, "bert_embeddings")

# Save model
model = SentenceTransformer(bert_model_name)
model.save(save_path)

print(f"BERT model saved to: {save_path}")


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

BERT model saved to: /content/drive/MyDrive/BioBERT/bert_embeddings
